In [1]:
import requests
from bs4 import BeautifulSoup

## 클리앙 알뜰구매 게시판

In [2]:
page_url = 'https://www.clien.net/service/board/jirum'
page = 1
params = {
    'po' : (page-1)
}

In [4]:
res = requests.get(page_url, params=params)
res

<Response [200]>

In [5]:
html = res.text
soup = BeautifulSoup(html, 'html.parser')

In [11]:
tag_list = soup.select('.contents_jirum .list_item')
len(tag_list)

30

In [12]:
tag = tag_list[0]

In [15]:
title_tag = tag.select_one('[data-role=list-title-text]') # 그냥 바로 속성명을 찾을 때는 대괄호인것
title_tag['href']
#title_tag.text

'/service/board/jirum/13483369?od=T31&po=0&category=&groupCd='

In [17]:
from urllib.parse import urljoin

In [18]:
urljoin(page_url, title_tag['href']) # 개별 포스팅 주소

'https://www.clien.net/service/board/jirum/13483369?od=T31&po=0&category=&groupCd='

In [20]:
post_url = urljoin(page_url, title_tag['href'])
post_title = title_tag.text
print(post_title, post_url)

[티몬] 타워즈 타이파이터 서큘레이터 (9,999원) https://www.clien.net/service/board/jirum/13483369?od=T31&po=0&category=&groupCd=


In [21]:
author_id = tag['data-author-id']
board_sn = tag['data-board-sn']
comment_count = tag['data-comment-count']
print(author_id, board_sn, comment_count)

gms010 13483369 17


In [28]:
try : 
    nickname = tag.select_one('.nickname img')['alt'] # 이미지가 들어간 것들을 처리하기 위한 처리
except TypeError:
    nickname = tag.select_one('.nickname').text.strip()
print(nickname)

국내


In [30]:
hit = int(tag.select_one('.hit').text)
hit

2806

In [32]:
from datetime import datetime

In [33]:
created_at_string = tag.select_one('.timestamp').text
created_at_string

'2019-05-12 17:20:00'

In [38]:
created_at = datetime.strptime(created_at_string, '%Y-%m-%d %H:%M:%S') # 문자를 날짜 정보로 변환
created_at

datetime.datetime(2019, 5, 12, 17, 20)

In [49]:
post_list = []
for tag in tag_list:
    title_tag = tag.select_one('[data-role=list-title-text]')
    post_url = urljoin(page_url, title_tag['href'])
    post_title = title_tag.text
    
    author_id = tag['data-author-id']
    board_sn = tag['data-board-sn']
    comment_count = tag['data-comment-count']
    try : 
        nickname = tag.select_one('.nickname img')['alt'] 
    except TypeError:
        nickname = tag.select_one('.nickname').text.strip()
    
    hit_string = tag.select_one('.hit').text
    
    if hit_string.endswith('k'): # 중간에 K로 끝나는 부분이 있어 오류가 나서 처리하는 코드
        hit = float(hit_string[:-1]) * 1000
    else:
        hit = int(hit_string)
    #print(hit_string, hit)
    
    created_at_string = tag.select_one('.timestamp').text
    created_at = datetime.strptime(created_at_string, '%Y-%m-%d %H:%M:%S')
    
    
    post_list.append({
        'post_url' : post_url,
        'post_title' : post_title,
        'author_id' : author_id,
        'board_sn' : board_sn,
        'comment_count' : comment_count,
        'nickname' : nickname,
        'hit' : hit,
        'created_at' : created_at,
    })
    
    
    
post_list[0]

{'post_url': 'https://www.clien.net/service/board/jirum/13483369?od=T31&po=0&category=&groupCd=',
 'post_title': '[티몬] 타워즈 타이파이터 서큘레이터 (9,999원)',
 'author_id': 'gms010',
 'board_sn': '13483369',
 'comment_count': '17',
 'nickname': '국내',
 'hit': 2806,
 'created_at': datetime.datetime(2019, 5, 12, 17, 20)}

특정게시물의 댓글 가져오기<br>
댓글은 페이지 소스 보기 해서 보면 없다<br>
그러면 네트워크 탭에서 XHR에서 새로고침해보면 commnet를 http 요청을 통해 받아 옴을 알수 있다.<br>
새탭으로 띄어서 보자<br>

## 댓글

In [51]:
comment_url='https://www.clien.net/service/board/jirum/13483369/comment'
params = 'order=date&po=0&ps=100&writer=gms010&webTag=ST03'
params.split('&')

['order=date', 'po=0', 'ps=100', 'writer=gms010', 'webTag=ST03']

webTag 는 없어도 되는 것 같고<br>
작성자가 댓글을 달면 메타 정보가 writer 인데 우리는 필요 없다.<br>
po는 페이징이고 ps는 몇개단위로 보여질것인지

In [53]:
comment_url='https://www.clien.net/service/board/jirum/13483369/comment'
params = {
    'order' : 'date',
    'po' : 0,
    'ps' : 100
}

In [56]:
post_url = 'https://www.clien.net/service/board/jirum/13483369?od=T31&po=0&category=&groupCd='
comment_list_url = post_url.split('?', 1)[0] + '/comment'

In [57]:
post_list = []
for tag in tag_list:
    title_tag = tag.select_one('[data-role=list-title-text]')
    post_url = urljoin(page_url, title_tag['href'])
    comment_list_url = post_url.split('?', 1)[0] + '/comment'
    post_title = title_tag.text
    
    author_id = tag['data-author-id']
    board_sn = tag['data-board-sn']
    comment_count = tag['data-comment-count']
    try : 
        nickname = tag.select_one('.nickname img')['alt'] 
    except TypeError:
        nickname = tag.select_one('.nickname').text.strip()
    
    hit_string = tag.select_one('.hit').text
    
    if hit_string.endswith('k'): # 중간에 K로 끝나는 부분이 있어 오류가 나서 처리하는 코드
        hit = float(hit_string[:-1]) * 1000
    else:
        hit = int(hit_string)
    #print(hit_string, hit)
    
    created_at_string = tag.select_one('.timestamp').text
    created_at = datetime.strptime(created_at_string, '%Y-%m-%d %H:%M:%S')
    
    
    post_list.append({
        'post_url' : post_url,
        'comment_list_url' : comment_list_url,
        'post_title' : post_title,
        'author_id' : author_id,
        'board_sn' : board_sn,
        'comment_count' : comment_count,
        'nickname' : nickname,
        'hit' : hit,
        'created_at' : created_at,
    })
    
post_list[0]

{'post_url': 'https://www.clien.net/service/board/jirum/13483369?od=T31&po=0&category=&groupCd=',
 'comment_list_url': 'https://www.clien.net/service/board/jirum/13483369/comment',
 'post_title': '[티몬] 타워즈 타이파이터 서큘레이터 (9,999원)',
 'author_id': 'gms010',
 'board_sn': '13483369',
 'comment_count': '17',
 'nickname': '국내',
 'hit': 2806,
 'created_at': datetime.datetime(2019, 5, 12, 17, 20)}

In [59]:
comment_list_url = post_url.split('?', 1)[0] + '/comment'
params = {
    'order' : 'date',
    'po' : 0,
    'ps' : 100
}
res = requests.get(comment_list_url, params=params)
res

<Response [200]>

In [60]:
html = res.text
soup = BeautifulSoup(html, 'html.parser')

In [61]:
tag_list = soup.select('.comment_row')
len(tag_list)

14

In [62]:
tag = tag_list[0]

In [63]:
author_id = tag['data-author-id']
comment_id = tag['data-comment-sn']

In [64]:
try : 
    nickname = tag.select_one('.nickname img')['alt'] 
except TypeError:
    nickname = tag.select_one('.nickname').text.strip()

In [67]:
content = tag.select_one('.comment_content').text.strip()
content

'감사합니다'

In [70]:
comment_list=[]
for tag in tag_list:
    author_id = tag['data-author-id']
    comment_id = tag['data-comment-sn']
    try : 
        nickname = tag.select_one('.nickname img')['alt'] 
    except TypeError:
        nickname = tag.select_one('.nickname').text.strip()
    content = tag.select_one('.comment_content').text.strip()
    comment_list.append({
        'author_id' : author_id,
        'comment_id' : comment_id,
        'nickname' : nickname,
        'content' : content,        
    })
comment_list[0]

{'author_id': 'numseal',
 'comment_id': '101047470',
 'nickname': '넘실이',
 'content': '감사합니다'}